### Дипломная работа Александра Соколова

#### Градиентный бустинг
Кернел 3 из 5 в разделе ML (отредактирован 21.04.2021)
---

# 1. Импорт библиотек, инициализация глобальных констант
## 1.1. Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
import tqdm
import pickle

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score
from sklearn.inspection import permutation_importance

import lightgbm as lgb
import catboost as cb

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import shap

np.warnings.filterwarnings('ignore')

## 1.2. Глобальные константы

In [ ]:
# CURRENT_DIR = './'  # имя текущей директории для локальной машины 
CURRENT_DIR = '../'  # имя текущей директории для каггл

PATH_TO_WORKDIR = CURRENT_DIR + 'working/'

In [ ]:
!pip freeze > requirements.txt

# 2. Импорт предобработанных данных
---
предобработка осуществлялась в [первом кернеле](https://www.kaggle.com/sokolovaleks/sf-dst-10-diplom-1-ml-sokolov)

In [ ]:
merged_train_data = pd.read_csv('../input/alfabattle2-sandbox/preproc_data_for_boosting/preproc_data_for_boosting/merged_data.csv')
merged_test_data = pd.read_csv('../input/alfabattle2-sandbox/preproc_data_for_boosting/preproc_data_for_boosting/merged_test_data.csv')

# 3. Разбиваем тренировочную выборку
---
Важно выделять валидационную выборку, чтобы контролировать обучение и не переобучаться. 

In [ ]:
targets = merged_train_data.flag.values

cv = KFold(n_splits=5, random_state=100, shuffle=True)

# 4. LightGBM + CV (1)
---
## 4.1. Первая модель (model1)

In [ ]:
num_model = 1

In [ ]:
!mkdir model1

In [ ]:
features = [x for x in merged_train_data.columns if x not in ['app_id', 'flag']]

In [ ]:
%%time

oof = np.zeros(len(merged_train_data))
train_preds = np.zeros(len(merged_train_data))

models = []

tree_params = {
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'max_depth': 3,
    'reg_lambda': 1,
    'num_leaves': 64,
    'n_jobs': 5,
    'n_estimators': 1000
}

for fold_, (train_idx, val_idx) in enumerate(cv.split(merged_train_data, targets), 1):
    print(f'Началось обучение на фолде номер:= {fold_}.')
    lgb_model = lgb.LGBMClassifier(**tree_params)
    train, val = merged_train_data.iloc[train_idx], merged_train_data.iloc[val_idx]
    
    lgb_model.fit(train[features], train.flag.values, eval_set=[(val[features], val.flag.values)],
              early_stopping_rounds=50, verbose=50)

    oof[val_idx] = lgb_model.predict_proba(val[features])[:, 1]
    train_preds[train_idx] += lgb_model.predict_proba(train[features])[:, 1] / (cv.n_splits-1)
    models.append(lgb_model)
    
    file_name_model = f'model{num_model}/model{num_model}_{fold_}.txt'
    lgb_model.booster_.save_model(PATH_TO_WORKDIR+file_name_model)
    print(f'Обучение на фолде номер:= {fold_} завершилось.')

In [ ]:
file_name_pickle = PATH_TO_WORKDIR + f'model{num_model}/feats_model{num_model}.pickle'
with open(file_name_pickle, 'wb') as f:
    pickle.dump(features, f)

In [ ]:
f'Train roc-auc model{num_model}', roc_auc_score(targets, train_preds)

In [ ]:
f'CV roc-auc model{num_model}', roc_auc_score(targets, oof)

## 4.2. Submission (model1) 

In [ ]:
score = np.zeros(len(merged_test_data))

for model in tqdm.tqdm_notebook(models):
    score += model.predict_proba(merged_test_data[features])[:, 1] / len(models)
    
submission = pd.DataFrame({
    'app_id' : merged_test_data.app_id.values,
    'score': score
}) 

submission.to_csv(f'sub_model{num_model}.csv', index=None)  # ~ 0.737 roc-auc на public test

# 5. Feature Importance
---
Оценка важности признаков - важный шаг в построении моделей. LightGBM имеет внутренние способы оценки важности признаков - на основе того, как часто делается сплит в вершине по признаку (split) и на основе того, какой суммарный прирост в информации дает разбиение по признаку (gain). Используем первый способ оценки важности признаков. Затем отберем топ признаков по важности и построим на них новые модели.

In [ ]:
importances = np.zeros(len(features))
for model in models:
    importances += model.feature_importances_ / len(models)

In [ ]:
def plot_feature_importance(feature_names, feature_scores, top_feats=20, title='Importance', x_label='Importance', 
                            y_label='Features'):
    tuples = [(name, round(score, 3)) for name, score in zip(feature_names, feature_scores)]
    tuples = sorted(tuples, key=lambda x: x[1])[-top_feats:]
    
    labels, values = zip(*tuples)
    _, ax = plt.subplots(1, 1, figsize=(10, 8))
    ylocs = np.arange(len(values))
    
    ax.barh(ylocs, values, align='center', height=0.4)
    for x, y in zip(values, ylocs):
        ax.text(x + 1, y, x, va='center')
        
    ax.set_yticks(ylocs)
    ax.set_yticklabels(labels)
    xlim = (0, max(values) * 1.1)
    ax.set_xlim(xlim)
    ylim = (-1, len(values))
    ax.set_ylim(ylim)
    
    ax.set_title(title)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.grid(True)

In [ ]:
plot_feature_importance(features, importances, top_feats=20, title='LightGBM feature importance', 
                        x_label='Importance')

In [ ]:
tuples = [(name, round(score, 3)) for name, score in zip(features, importances)]
tuples = sorted(tuples, key=lambda x: x[1])

top_selected_feats = [x[0] for x in tuples if x[1] >= np.median(importances)]
print(f'Кол-во отобранных признаков на этапе Feature importance (median):= {len(top_selected_feats)}')

# 6. LightGBM + CV (2)
---
## 6.1. Вторая модель (model2)
(на признаках расчетная важность которых не меньше медианы)

In [ ]:
num_model = 2

In [ ]:
!mkdir model2

In [ ]:
%%time

oof = np.zeros(len(merged_train_data))
train_preds = np.zeros(len(merged_train_data))

new_models = []

tree_params = {
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'max_depth': 3,
    'reg_lambda': 1,
    'num_leaves': 64,
    'n_jobs': 5,
    'n_estimators': 1000
}


for fold_, (train_idx, val_idx) in enumerate(cv.split(merged_train_data, targets), 1):
    print(f'Началось обучение на фолде номер:= {fold_}.')
    lgb_model = lgb.LGBMClassifier(**tree_params)
    train, val = merged_train_data.iloc[train_idx], merged_train_data.iloc[val_idx]
    
    lgb_model.fit(train[top_selected_feats], train.flag.values, eval_set=[(val[top_selected_feats], val.flag.values)],
              early_stopping_rounds=50, verbose=50)

    oof[val_idx] = lgb_model.predict_proba(val[top_selected_feats])[:, 1]
    train_preds[train_idx] += lgb_model.predict_proba(train[top_selected_feats])[:, 1] / (cv.n_splits-1)
    new_models.append(lgb_model)
    
    file_name_model = f'model{num_model}/model{num_model}_{fold_}.txt'
    lgb_model.booster_.save_model(PATH_TO_WORKDIR+file_name_model)
    print(f'Обучение на фолде номер:= {fold_} завершилось.')

In [ ]:
file_name_pickle = PATH_TO_WORKDIR + f'model{num_model}/feats_model{num_model}.pickle'
with open(file_name_pickle, 'wb') as f:
    pickle.dump(top_selected_feats, f)

In [ ]:
f'Train roc-auc model{num_model}', roc_auc_score(targets, train_preds)

In [ ]:
f'CV roc-auc model{num_model}', roc_auc_score(targets, oof)

In [ ]:
score = np.zeros(len(merged_test_data))

for model in tqdm.tqdm_notebook(new_models):
    score += model.predict_proba(merged_test_data[top_selected_feats])[:, 1] / len(models)
    
submission = pd.DataFrame({
    'app_id' : merged_test_data.app_id.values,
    'score': score
}) 

submission.to_csv(f'sub_model{num_model}.csv', index=None)  # ~ 0.7346 на public test

# 7. Permutation importance
---
Это способ оценки важности признаков, который можно применить к любой обученной модели на табличных данных. Данный тип важности определеяется для каждого признака, как изменение в скоре (в нашем случае - roc_auc) при случайном перемешиваниии столбца с значениями этого признака несколько раз.

In [ ]:
%%time
permut_importance = np.zeros(len(features))

for fold_, (_, val_idx) in enumerate(cv.split(merged_train_data, targets), 1):
    print(f'Началась обработка фолда:= {fold_}')
    val = merged_train_data.iloc[val_idx]
    importances_report = permutation_importance(models[fold_-1], val[features], val.flag.values, n_repeats=3, 
                                     n_jobs=3, scoring='roc_auc')
    permut_importance += importances_report.importances_mean / cv.n_splits
    print(f'Обработка фолда:= {fold_} закончена.')

In [ ]:
plot_feature_importance(features, permut_importance*10000, top_feats=20, title='Permutation importance', 
                        x_label='Importance')

Используя техники оценки важности признаков можно интерпретировать модель, пытаться генерировать новые признаки или упрощать модель, удаляя самые неинформативные. Попробуем оставить топ информативных признаков (с точки зрения) `permutation importance`. Удаление из модели признаков, как правило, уменьшает ее склонность к переобучению. А самое главное - может существенно ускорить процесс обучения и последующий процесс использования ее в продуктовых целях, что для меня достаточно важно потому что есть цель развернуть деплой модели на heroku, а его мощности скромные. 

In [ ]:
tuples = [(name, round(score, 3)) for name, score in zip(features, permut_importance)]
tuples = sorted(tuples, key=lambda x: x[1])

top_selected_feats = [x[0] for x in tuples if x[1] > 0]
print(f'Кол-во отобранных признаков после метода Permutation importance:= {len(top_selected_feats)}')

# 8. LightGBM + CV (3)
---
## 8.1. Третья модель (model3)
(на признаках после отбора с помощью методов Features importance(median) и Permutation importance)

In [ ]:
num_model = 3

In [ ]:
!mkdir model3

In [ ]:
%%time

oof = np.zeros(len(merged_train_data))
train_preds = np.zeros(len(merged_train_data))

new_models = []

tree_params = {
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'max_depth': 3,
    'reg_lambda': 1,
    'num_leaves': 64,
    'n_jobs': 5,
    'n_estimators': 1000
}


for fold_, (train_idx, val_idx) in enumerate(cv.split(merged_train_data, targets), 1):
    print(f'Training with fold {fold_} started.')
    lgb_model = lgb.LGBMClassifier(**tree_params)
    train, val = merged_train_data.iloc[train_idx], merged_train_data.iloc[val_idx]
    
    lgb_model.fit(train[top_selected_feats], train.flag.values, eval_set=[(val[top_selected_feats], val.flag.values)],
              early_stopping_rounds=50, verbose=50)

    
    oof[val_idx] = lgb_model.predict_proba(val[top_selected_feats])[:, 1]
    train_preds[train_idx] += lgb_model.predict_proba(train[top_selected_feats])[:, 1] / (cv.n_splits-1)
    new_models.append(lgb_model)
    
    file_name_model = f'model{num_model}/model{num_model}_{fold_}.txt'
    lgb_model.booster_.save_model(PATH_TO_WORKDIR+file_name_model)
    print(f'Training with fold {fold_} completed.')

In [ ]:
file_name_pickle = PATH_TO_WORKDIR + f'model{num_model}/feats_model{num_model}.pickle'
with open(file_name_pickle, 'wb') as f:
    pickle.dump(top_selected_feats, f)

In [ ]:
f'Train roc-auc model{num_model}', roc_auc_score(targets, train_preds)

In [ ]:
f'CV roc-auc model{num_model}', roc_auc_score(targets, oof)

In [ ]:
score = np.zeros(len(merged_test_data))

for model in tqdm.tqdm_notebook(new_models):
    score += model.predict_proba(merged_test_data[top_selected_feats])[:, 1] / len(models)
    
submission = pd.DataFrame({
    'app_id' : merged_test_data.app_id.values,
    'score': score
}) 

submission.to_csv(f'sub_model{num_model}.csv', index=None)  # ~ 0.736 roc-auc на public test

# 9. Интерпретация предсказаний модели (shap)
---
В задаче кредитного скорринга важна интепретируемость модели, для этого можно использовать  библиотеку shap. Мы используем так называемый summary plot, позволяющий получить общую картину о влиянии того или иного признака на предсказание модели визуализуруя всю выборку сразу. Я не смог разобраться как использовать методы shap для модели обученной на 5-ти фолдах. Ниже я заново обучаю модель после сплита 90:10 без деления на фолды, чтобы выделить основные закономерности влияния фич на таргет и попытаться визуализировать их разделяющую способность.

In [ ]:
%%time
train, val = train_test_split(merged_train_data, random_state=100, test_size=0.1)

tree_params = {
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'max_depth': 3,
    'reg_lambda': 1,
    'num_leaves': 64,
    'seed': 100,
    'n_jobs': 5,
    'n_estimators': 1000
}

lgb_model = lgb.LGBMClassifier(**tree_params)
lgb_model.fit(train[features], train.flag.values, eval_set=[(val[features], val.flag.values)],
              early_stopping_rounds=50, verbose=-1)

In [ ]:
explainer = shap.TreeExplainer(lgb_model)

In [ ]:
shap_values = explainer.shap_values(val[features])

In [ ]:
shap.summary_plot(shap_values[1], val[features])

График выше показывает влияние и разделяющую способность признаков:
- каждая точка графика это клиент
- цвет - значение прогноза дефолта, чем краснее - тем прогноз дефолта выше, и наоборот
- горизонтальное положение точки показывает, приводит ли значение конкретной фичи этого клиента к росту предсказания дефолта, или наоборот.

***Например можно увидеть следующие тренды:***

1. уменьшение признака 'hour_diff_median' приводит к росту значения целевой переменной (то есть можно предположить, что клиенту приходится делать транзакции слишком часто - а это свидетельство того, что клиент вероятнее выйдет в дефолт, чем клиент который чувствует себя более уверенно в финансовом плане и может позволить делать крупные оплаты сразу не разбивая их на части)
2. увеличение признака 'hour_diff_max' приводит к росту значения целевой переменной (то есть можно предположить, что клиент долго не пользовался картой это могло произойти в следствии овердрафта и необходимости перекредитования в другом банке)
3. уменьшение признака 'count_mcc_category_9' приводит к росту значения целевой переменной (к сожалению категории торговых точек как данные о клиентах были обезличены, остается только предположить, что вероятно это категория 'Кафе,бары и рестораны' и клиенты испытывающие сложности в финансах перестают их посещать или делают это существенно реже.)
4. увеличение признака 'count_operation_type_5' приводит к росту значения целевой переменной (из-за обезличивания типа операций и дополнительного анализа видно что у большинства клиентов таких операций нет, вероятно речь об операции внесение минимального платежа, вместо гашения очередного платежа. И собственно если это так что увеличение кол-ва таких операций приводит к дефолту потому что нагрузка в этом случае только возрастает)
5. среднее и максимальное значение признака 'product' приводит к росту значения целевой переменной (предположу что 0 - кредит безналичными на карту, 1 - это кредит наличными, 2 - кредит на крупные покупки, 3 - кредит на покупку авто, 4 - ипотека. Сделать такое предположение можно и-за того что 80% всех кредитов по кол-ву приходится на продукты 0,1. А самый редкий это продукт 4. Среднее значение дефолтов 2-2,5% процента по продуктам 0,1,3. По продукту 2 - 7%. А по продукту 4 - 3%. Вероятно кредиты на крупные покупки проверяются менее чем ипотека и они не имеют обеспечения как авто для погашения кредита.)
6. уменьшение признака 'days_before_max' приводит к росту значения целевой переменной (низкое значение максимума кол-ва дней перед взятием кредита означает короткая история транзакций в банке. Можно предположить, что клиент открыл карту и через короткое время получил кредит. Это либо мошеническая схема, но этот вариант мы не рассматривает ибо он скорее всего не массовый. А вероятно это случай перекредитования в другом банке, когда банк по соновной карте перестал предоставлять или увеличивать кредитную линию. Что является свидетельством неплатежеспособности клиента.)

# 10. Target Permutation
---
Отбор признаков методом target permutation основан на сравнении важности признака, которая быал получена при обучении на обычной выборке, с распределением важности этого признака для моделей, которые обучены на выборках с перемешанной целевой переменной.

In [ ]:
def get_feature_importances(frame, shuffle=False, seed=100, importance_type='gain'):
    # запишем целевую target в переменную
    y = frame.flag.values.copy()
    if shuffle:
        # перемешаем целевую переменную
        y = frame.flag.copy().sample(frac=1.0).values
    
    # обучим случайный лес из LightGBM ( реализация гораздо быстрее, чем в sklearn )
    # сделаем это для того, чтобы не выделять отдельную выборку под валидацию для градиентного бустинга
    tree_params = {
    'objective': 'binary',
    'boosting_type': 'rf',
    'metric': 'auc',
    'subsample_freq': np.random.choice([5, 10, 15, 20]),
    'subsample': np.random.choice([0.6, 0.8, 0.9]),
    'max_depth': np.random.choice([7, 8, 10]),
    'num_leaves': 64,
    'seed': seed,
    'n_jobs': 5,
    'n_estimators': 100,
    'importance_type': importance_type
    }
    
    lgb_model = lgb.LGBMClassifier(**tree_params)
    lgb_model.fit(frame[features], y)
    
    importance = pd.DataFrame({
        'feature': features,
        f'importance_{importance_type}': lgb_model.feature_importances_,
        'score': roc_auc_score(y, lgb_model.predict_proba(frame[features])[:, 1])
    })
    
    return importance

Посчитаем важность признаков (на основе количества сплитов по признаку в алгоритме random forest) при обучении на исходной выборке

In [ ]:
real_importance = get_feature_importances(train, shuffle=False, seed=100, importance_type='split')
real_importance.head()

Посчитаем важность признаков для выборок, у которых целевая переменная перемешана. То есть фактически, обучаем алгоритм на шуме.

In [ ]:
def calculate_null_importance(frame, num_runs=80, importance_type='gain'):
    null_importance = pd.DataFrame()
    for i in tqdm.tqdm_notebook(range(num_runs)):
        # посчитаем текущую важность признаков при перемешивании целевой переменной
        importance = get_feature_importances(frame, shuffle=True, seed=None, importance_type=importance_type)
        importance['run'] = i + 1 
        null_importance = pd.concat([null_importance, importance], axis=0)
    return null_importance

In [ ]:
%%time
null_importance = calculate_null_importance(train, num_runs=30, importance_type='split')

Теперь имеем возможность для каждого признака построить гистограмму важности этого признака при обучении на выборке с перемешанным таргетом и на выборке с реальной целевой переменной. В идеале, гистограмма должна быть как можно дальше от реального значения важности признака

In [ ]:
def plot_distribution(real_importance, null_importance, feature_name, importance_name='gain'):
    fig, ax = plt.subplots(figsize=(8, 6))
    a = ax.hist(null_importance.loc[null_importance['feature'] == feature_name, f'importance_{importance_name}'].values, 
                label='Null importances')
    
    ax.vlines(x=real_importance.loc[real_importance['feature'] == feature_name, f'importance_{importance_name}'].mean(), 
               ymin=0, ymax=np.max(a[0]), color='r',linewidth=10, label='Real Target')
    ax.legend()
    ax.set_title(f'Split Importance of {feature_name}', fontweight='bold')
    plt.xlabel(f'Null Importance ({importance_name}) Distribution for {feature_name}')

In [ ]:
plot_distribution(real_importance, null_importance, feature_name='product', importance_name='split')

In [ ]:
plot_distribution(real_importance, null_importance, feature_name='hour_diff_median', importance_name='split')

In [ ]:
plot_distribution(real_importance, null_importance, feature_name='amnt_sum', importance_name='split')

Попробуем поставить каждому признаку новый скор, важность, с учетом той информации, которую имеем. Один из вариантов - поделить реальноа значение важности признака на среднее значение для перемешанных важностей этого признака. 

In [ ]:
scores = []
importance_name = 'split'

for feature in features:
    hist_null_importance = null_importance.loc[null_importance['feature'] == feature, f'importance_{importance_name}'].values
    actual_importance = real_importance.loc[real_importance['feature'] == feature, f'importance_{importance_name}'].mean()
    score = (1e-10 + actual_importance) / (1 + np.mean(hist_null_importance))        
    scores.append(round(score, 3))

In [ ]:
plot_feature_importance(features, scores, top_feats=20, title='target permutation importance', 
                        x_label='Importance')

In [ ]:
tuples = [(name, round(score, 3)) for name, score in zip(features, scores)]
tuples = sorted(tuples, key=lambda x: x[1])

top_selected_feats = [x[0] for x in tuples if x[1] > 0]
print(f'Кол-во отобранных признаков после всех методов отбора:= {len(top_selected_feats)}')

# 11. LightGBM + CV (4)
---
## 11.1. Четвертая модель (model4)
(на признаках после отбора с помощью всех методов Features importance(median), Permutation importance и Target importance)

In [ ]:
num_model = 4

In [ ]:
!mkdir model4

In [ ]:
%%time

oof = np.zeros(len(merged_train_data))
train_preds = np.zeros(len(merged_train_data))

new_models = []

tree_params = {
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'max_depth': 3,
    'reg_lambda': 1,
    'num_leaves': 64,
    'n_jobs': 5,
    'n_estimators': 1000
}


for fold_, (train_idx, val_idx) in enumerate(cv.split(merged_train_data, targets), 1):
    print(f'Training with fold {fold_} started.')
    lgb_model = lgb.LGBMClassifier(**tree_params)
    train, val = merged_train_data.iloc[train_idx], merged_train_data.iloc[val_idx]
    
    lgb_model.fit(train[top_selected_feats], train.flag.values, eval_set=[(val[top_selected_feats], val.flag.values)],
              early_stopping_rounds=50, verbose=50)

    
    oof[val_idx] = lgb_model.predict_proba(val[top_selected_feats])[:, 1]
    train_preds[train_idx] += lgb_model.predict_proba(train[top_selected_feats])[:, 1] / (cv.n_splits-1)
    new_models.append(lgb_model)
    
    file_name_model = f'model{num_model}/model{num_model}_{fold_}.txt'
    lgb_model.booster_.save_model(PATH_TO_WORKDIR+file_name_model)
    print(f'Training with fold {fold_} completed.')

In [ ]:
file_name_pickle = PATH_TO_WORKDIR + f'model{num_model}/feats_model{num_model}.pickle'
with open(file_name_pickle, 'wb') as f:
    pickle.dump(top_selected_feats, f)

In [ ]:
f'Train roc-auc model{num_model}', roc_auc_score(targets, train_preds)

In [ ]:
f'CV roc-auc model{num_model}', roc_auc_score(targets, oof)

In [ ]:
score = np.zeros(len(merged_test_data))

for model in tqdm.tqdm_notebook(new_models):
    score += model.predict_proba(merged_test_data[top_selected_feats])[:, 1] / len(models)
    
submission = pd.DataFrame({
    'app_id' : merged_test_data.app_id.values,
    'score': score
})
submission.to_csv(f'sub_model{num_model}.csv', index=None)  # ~ 0.733 на public test

Расчет модели Catboost реализован в следующем кернеле, чтобы не использовать квоту так как запускается на GPU

**Результаты всех экспериментов моделей (метрика AUC ROC):**  
                    Model                            | число призн. |    Train     |     CV      |   Public Test      
---  
                     lgb+cv(5)                     |        127           |    0.801    |    0.77     |    0.737       
       lgb+cv(5)+split_importance      |          64            |    0.795    |    0.767    |    0.735  
lgb+cv(5)+permutation_importance |         57            |    0.796    |    0.768    |    0.736   
    lgb+cv(5)+target_permutation     |          60            |    0.796    |    0.766    |    0.733 
